### DATA PT12 - Proyecto final - Grupo 2
#### Control de calidad de los datos del archivo Google Maps review de los estados de New Jersey (NJ) y New York (NY)

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Obtener la lista de archivos JSON en carpeta
archivos_json = glob.glob(os.path.join("datos/googlemaps/New_Jersey", "*.json"))

In [22]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
for archivo in archivos_json:
    df = pd.read_json(archivo, lines=True)
    df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl

In [18]:
# Carga los archivos Pickle generados y los combina en un solo DataFrame.
df_list = [pd.read_pickle(archivo.replace(".json", ".pkl")) for archivo in archivos_json] # convierte el nombre de cada archivo de .json a .pkl para cargarlo correctamente.
df_rev_NJ = pd.concat(df_list, ignore_index=True)

df_rev_NJ.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.011903e+20,Monica Zobbi,1614634731992,5,Viktoriya is always very professional and atte...,None,None,0x89c2fb42465dd457:0x437fc2232abf899
1,1.172699e+20,Vitaliia Stefurak,1572974792541,5,"If I could leave them 10 starts I would , unbe...",None,None,0x89c2fb42465dd457:0x437fc2232abf899
2,1.152270e+20,Maria Guzman,1578525387607,5,Laura took her time to know my hair and what s...,None,None,0x89c2fb42465dd457:0x437fc2232abf899
3,1.161828e+20,Jo Rad,1602103279588,5,"Wow, what an experience! Haircut by Victoria! ...",None,"{'time': 1576097102614, 'text': 'Thank you! Th...",0x89c2fb42465dd457:0x437fc2232abf899
4,1.010712e+20,Kimberly Hults,1593034107348,5,Love this salon! The great experience starts o...,None,"{'time': 1575147680996, 'text': 'Thanks for th...",0x89c2fb42465dd457:0x437fc2232abf899


In [19]:
df_rev_NJ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950000 entries, 0 to 1949999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 119.0+ MB


In [20]:
# Eliminamos, por ahora, pics y resp. no usaremos esas columnas
df_rev_NJ = df_rev_NJ.drop(columns=['pics', 'resp'])

In [21]:
# # reviso si hay reviews duplicadas
df_dup = df_rev_NJ[df_rev_NJ.duplicated(subset=['user_id', 'gmap_id', 'time'], keep=False)]
df_dup = df_dup.sort_values(['user_id', 'gmap_id', 'time'])

print(df_dup)

              user_id             name           time  rating  \
1135631  1.000011e+20       Paul Falzo  1571439140777       5   
1135644  1.000011e+20       Paul Falzo  1571439140777       5   
1280946  1.000034e+20      Scuba Steve  1519592599394       4   
1280955  1.000034e+20      Scuba Steve  1519592599394       4   
1351283  1.000034e+20     Joshua Cohen  1484612006694       4   
...               ...              ...            ...     ...   
879607   1.184462e+20    Michael Bloom  1595261023187       5   
939359   1.184467e+20  Brandon Tedesco  1486004605968       3   
939368   1.184467e+20  Brandon Tedesco  1486004605968       3   
1675824  1.184467e+20  Brandon Tedesco  1486005838053       3   
1675845  1.184467e+20  Brandon Tedesco  1486005838053       3   

                                                      text  \
1135631                       All good stuff and pricing..   
1135644                       All good stuff and pricing..   
1280946       Beautiful mall. Alw

Hay 29842 reviews duplicadas por las columnas 'user_id', 'gmap_id', 'time' para el estado de NJ. Las elimino

In [22]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_NJ_sindup = df_rev_NJ.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [23]:
# rename column name to user_name
df_rev_NJ_sindup.rename(columns={'name': 'user_name'}, inplace=True)

In [24]:
# convertir columna time en int64 a datetime
df_rev_NJ_sindup['date'] = pd.to_datetime(df_rev_NJ_sindup['time'], unit='ms')

In [25]:
# Eliminamos la columan time
df_rev_NJ_sindup = df_rev_NJ_sindup.drop(columns=['time'])
df_rev_NJ_sindup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1919845 entries, 0 to 1949914
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    float64       
 1   user_name  object        
 2   rating     int64         
 3   text       object        
 4   gmap_id    object        
 5   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 102.5+ MB


In [26]:
# guardo df_rev_NJ_sindup
df_rev_NJ_sindup.to_parquet('datos/googlemaps/df_rev_NJ_sindup202502110757.parquet' , engine='fastparquet')

In [27]:
# leo el archivo de las pizzerias mt_pizza_NJNY para hacer join con este
# How to read a Parquet file into Pandas DataFrame?
df_mt_pizza_NJNY = pd.read_parquet('datos/googlemaps/df_mt_pizza_NJNY_202502081554.parquet' , engine='fastparquet')

In [38]:
df_mt_pizza_NJNY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252 entries, 0 to 3251
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         3252 non-null   object 
 1   business_name   3252 non-null   object 
 2   address         3239 non-null   object 
 3   city            3252 non-null   object 
 4   state           3252 non-null   object 
 5   zip_code        3252 non-null   object 
 6   latitude        3252 non-null   float64
 7   longitude       3252 non-null   float64
 8   avg_rating      3252 non-null   float64
 9   num_of_reviews  3252 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 254.2+ KB


In [39]:
df_mt_pizza_NJNY.to_excel("datos/googlemaps/df_mt_pizza_NJNY.xlsx")

In [28]:
# cuento la cantidad de pizzerias en cada estado / no puedo contar con una sola columna
df_mt_pizza_NJNY[['state', 'business_name' ]].groupby('state').count()

,business_name
state,
NJ,1134
NY,2118


Googlemaps tiene datos de 1134 pizzerias en NJ y 2118 pizzerias en NY

In [29]:
# separo las pizzerias de NY de las de NJ. Luego hago left join con las reviews y vuelvo a unir los dataframe
df_mt_pizza_NJ = df_mt_pizza_NJNY[df_mt_pizza_NJNY['state'] == 'NJ']
df_mt_pizza_NJ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1134 entries, 6 to 3247
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         1134 non-null   object 
 1   business_name   1134 non-null   object 
 2   address         1130 non-null   object 
 3   city            1134 non-null   object 
 4   state           1134 non-null   object 
 5   zip_code        1134 non-null   object 
 6   latitude        1134 non-null   float64
 7   longitude       1134 non-null   float64
 8   avg_rating      1134 non-null   float64
 9   num_of_reviews  1134 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 97.5+ KB


In [30]:
df_mt_pizza_NY = df_mt_pizza_NJNY[df_mt_pizza_NJNY['state'] == 'NY']
df_mt_pizza_NY.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2118 entries, 0 to 3251
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         2118 non-null   object 
 1   business_name   2118 non-null   object 
 2   address         2109 non-null   object 
 3   city            2118 non-null   object 
 4   state           2118 non-null   object 
 5   zip_code        2118 non-null   object 
 6   latitude        2118 non-null   float64
 7   longitude       2118 non-null   float64
 8   avg_rating      2118 non-null   float64
 9   num_of_reviews  2118 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 182.0+ KB


Uno el dataframe de datos de pizzerias de NJ en un left join con el de los datos de reviews de NJ para obtener todas las reviews de pizzerias de NJ y agregarle el nombre del local. Mas adelante hare los mismo para los locales de NY. Y luego unire los dos dataframes

In [31]:
df_pizza_NJ_rev = pd.merge(df_mt_pizza_NJ, df_rev_NJ_sindup, how='left', on='gmap_id') # tengo que hacer left join

In [32]:
df_pizza_NJ_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54275 entries, 0 to 54274
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gmap_id         54275 non-null  object        
 1   business_name   54275 non-null  object        
 2   address         54237 non-null  object        
 3   city            54275 non-null  object        
 4   state           54275 non-null  object        
 5   zip_code        54275 non-null  object        
 6   latitude        54275 non-null  float64       
 7   longitude       54275 non-null  float64       
 8   avg_rating      54275 non-null  float64       
 9   num_of_reviews  54275 non-null  int64         
 10  user_id         54045 non-null  float64       
 11  user_name       54045 non-null  object        
 12  rating          54045 non-null  float64       
 13  text            32794 non-null  object        
 14  date            54045 non-null  datetime64[ns]
dtypes:

Hay 230 pizzerias del NJ que no tienen reviews en el archivo usado. Se eliminaran con las de los locales de NY que tampoco tengan reseñas. </br>

In [ ]:
# preparo lo de NY

In [33]:
# Obtener la lista de archivos JSON en carpeta
archivos_json = glob.glob(os.path.join("datos/googlemaps/New_York", "*.json"))

In [47]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
for archivo in archivos_json:
    df = pd.read_json(archivo, lines=True)
    df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl

In [34]:
# Carga los archivos Pickle generados y los combina en un solo DataFrame.
df_list = [pd.read_pickle(archivo.replace(".json", ".pkl")) for archivo in archivos_json] # convierte el nombre de cada archivo de .json a .pkl para cargarlo correctamente.
df_rev_NY = pd.concat(df_list, ignore_index=True)

df_rev_NY.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.137221e+20,Alvin Martinez,1603494795361,5,I'm late to posting this but this store especi...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c25fc9494dce47:0x6d63c807b59a55
1,1.072934e+20,Johnnie Jackson,1620157037403,1,Very dissatisfied I did not get my phone the p...,None,"{'time': 1620268360920, 'text': 'We pride ours...",0x89c25fc9494dce47:0x6d63c807b59a55
2,1.003786e+20,Manie Blazer,1597431662039,5,Excellent very well done with professional car...,None,None,0x89c25fc9494dce47:0x6d63c807b59a55
3,1.149982e+20,Fashion Fiinds,1543773862044,5,Basing my review strictly on the service I rec...,None,"{'time': 1543855317372, 'text': 'Thanks for th...",0x89c25fc9494dce47:0x6d63c807b59a55
4,1.171782e+20,Andres Rieloff,1597279097718,1,Bad! Disorganized. I'm being totally honest. I...,None,None,0x89c25fc9494dce47:0x6d63c807b59a55


In [35]:
df_rev_NY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB


In [36]:
# # reviso si hay reviews duplicadas
df_dup_NY = df_rev_NY[df_rev_NY.duplicated(subset=['user_id', 'gmap_id', 'time'], keep=False)]
df_dup_NY = df_dup_NY.sort_values(['user_id', 'gmap_id', 'time'])

print(df_dup_NY)

              user_id             name           time  rating  \
433190   1.000013e+20   Daisy Crawford  1562613047637       5   
433191   1.000013e+20   Daisy Crawford  1562613047637       5   
519864   1.000014e+20     Matt Solomon  1502904476201       5   
519873   1.000014e+20     Matt Solomon  1502904476201       5   
2417594  1.000015e+20  Anthony Puglisi  1550149671999       4   
...               ...              ...            ...     ...   
357900   1.184462e+20    Jonathan Fung  1569217448412       5   
1201332  1.184465e+20      Chinh Chinh  1529017525510       4   
1201335  1.184465e+20      Chinh Chinh  1529017525510       4   
1855121  1.184466e+20      Alec Holden  1584810613275       5   
1855122  1.184466e+20      Alec Holden  1584810613275       5   

                                                      text  \
433190   Close by authentic Chinese food. Place is not ...   
433191   Close by authentic Chinese food. Place is not ...   
519864                           

Hay 59.027 reseñas duplicadas para NY. Las elimino

In [37]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_NY_sindup = df_rev_NY.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [39]:
# convertir columna time en int64 a datetime
df_rev_NY_sindup['date'] = pd.to_datetime(df_rev_NY_sindup['time'], unit='ms')

In [40]:
# Eliminamos la columan time
df_rev_NY_sindup = df_rev_NY_sindup.drop(columns=['time'])
df_rev_NY_sindup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2640527 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype         
---  ------   -----         
 0   user_id  float64       
 1   name     object        
 2   rating   int64         
 3   text     object        
 4   pics     object        
 5   resp     object        
 6   gmap_id  object        
 7   date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 181.3+ MB


In [41]:
# Eliminamos, por ahora, pics y resp. no usaremos esas columnas
df_rev_NY_sindup = df_rev_NY_sindup.drop(columns=['pics', 'resp'])

In [42]:
# rename column name to user_name
df_rev_NY_sindup.rename(columns={'name': 'user_name'}, inplace=True)

Uno el dataframe de datos de pizzerias de NY en un left join con el de los datos de reviews de NY para obtener todas las reviews de pizzerias de NY y agregarle el nombre del local.

In [43]:
df_pizza_NY_rev = pd.merge(df_mt_pizza_NY, df_rev_NY_sindup, how='left', on='gmap_id') # tengo que hacer left join

In [44]:
df_pizza_NY_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97120 entries, 0 to 97119
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gmap_id         97120 non-null  object        
 1   business_name   97120 non-null  object        
 2   address         97062 non-null  object        
 3   city            97120 non-null  object        
 4   state           97120 non-null  object        
 5   zip_code        97120 non-null  object        
 6   latitude        97120 non-null  float64       
 7   longitude       97120 non-null  float64       
 8   avg_rating      97120 non-null  float64       
 9   num_of_reviews  97120 non-null  int64         
 10  user_id         96613 non-null  float64       
 11  user_name       96613 non-null  object        
 12  rating          96613 non-null  float64       
 13  text            57918 non-null  object        
 14  date            96613 non-null  datetime64[ns]
dtypes:

Hay 507 pizzerias del NY de las que no hay reviews en el archivo usado.

In [45]:
# concateno las reviews de googlemaps de las pizzerias de NY y NJ
df_pizza_NJNY_rev = pd.concat([df_pizza_NJ_rev, df_pizza_NY_rev], ignore_index=True)

In [58]:
df_pizza_NJNY_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151395 entries, 0 to 151394
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   gmap_id         151395 non-null  object 
 1   business_name   151395 non-null  object 
 2   address         151299 non-null  object 
 3   city            151395 non-null  object 
 4   state           151395 non-null  object 
 5   zip_code        151395 non-null  object 
 6   latitude        151395 non-null  float64
 7   longitude       151395 non-null  float64
 8   avg_rating      151395 non-null  float64
 9   num_of_reviews  151395 non-null  int64  
 10  user_id         150658 non-null  float64
 11  user_name       150658 non-null  object 
 12  time            150658 non-null  float64
 13  rating          150658 non-null  float64
 14  text            90712 non-null   object 
dtypes: float64(6), int64(1), object(8)
memory usage: 17.3+ MB


Hay 737 pizzerias de las que no hay reviews en el archivo usado. Esto se infiere porque hay esa cantidad de user_id nulos. </br>
Es decir que al hacer un left join no habia una una fila con datos de review para ese local. </br>
Esas reseñas deben corresponder a un periodo anterior al del que tenemos reseñas.

Se eliminarán las filas correspondientes a esos locales de este dataframe. </br>
Se mantendrán las filas con los datos de esos locales en el dataframe de pizzerias ya que indican cantidad de reseñas y avg_rating. </br>

In [46]:
# elimino las filas de los locales sin reseñas
df_pizza_NJNY_rev1 = df_pizza_NJNY_rev[df_pizza_NJNY_rev['user_id'].notna()]

In [47]:
df_pizza_NJNY_rev1.head()

,gmap_id,business_name,address,city,state,zip_code,latitude,longitude,avg_rating,num_of_reviews,user_id,user_name,rating,text,date
0,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,1.137158e+20,Creative Builders Llc,1.0,Went in there to pick up my sandwich that I or...,2018-01-24 04:25:16.239
1,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,1.138407e+20,Alex Sheppard,3.0,"This is a chicken shack in the hood,we call it...",2012-10-16 02:39:32.591
2,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,1.089485e+20,Yolanda Hoffman,5.0,Good food,2018-01-25 04:33:37.207
3,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,1.151836e+20,The Johnsons,5.0,Great pizza!!!,2016-06-15 01:12:51.022
4,0x89c3acab12c10d5d:0x73285fc4bd781796,Gulistan Pizza,783 S Orange Ave,Newark,NJ,07106,40.745207,-74.220116,3.3,8,1.131849e+20,Djaliil Sakande,5.0,Close,2018-02-02 00:21:38.584


In [48]:
# cuento la cantidad de reseñas de pizzerias en cada estado / no puedo contar con una sola columna
df_pizza_NJNY_rev1[['state', 'gmap_id' ]].groupby('state').count()

,gmap_id
state,
NJ,54045
NY,96613


Hay 54045 reseñas para locales del estado de NJ y 96613 reseñas para locales del estado de NY

In [49]:
# guardo df_pizza_NJNY_rev
df_pizza_NJNY_rev1.to_parquet('datos/googlemaps/df_pizza_NJNY_rev202502110804.parquet' , engine='fastparquet')